# Sentiment Analysis of COVID-19 Tweets: When did the Public Panic Set In?

<img src="wesley-tingey-DBu9IrkuPFM-unsplash.jpg">

# Imports

In [10]:
%matplotlib inline

# Generic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# Get JSON
import json

# Obtain Data

In [4]:
df = pd.read_csv("covid_tweets copy.csv")
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,matching_rules,retweeted_status,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,display_text_range,extended_tweet
0,Fri Jan 31 23:58:59 +0000 2020,1223395284119695360,1223395284119695360,No les miento....\nEse Coronavirus me tiene as...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Jan 31 23:58:59 +0000 2020,1223395283796746242,1223395283796746242,"RT @CNN: There have been more than 9,800 cases...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,[{'tag': None}],{'created_at': 'Fri Jan 31 23:30:08 +0000 2020...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Jan 31 23:58:59 +0000 2020,1223395283477897216,1223395283477897216,RT @CanSaludable: Preguntas y respuestas #coro...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,[{'tag': None}],{'created_at': 'Fri Jan 31 23:30:13 +0000 2020...,False,"{'media': [{'id': 1223388032843288578, 'id_str...",NaN,NaN,NaN,NaN,NaN,NaN
3,Fri Jan 31 23:58:59 +0000 2020,1223395283121442816,1223395283121442816,RT @sanidadgob: El Centro Nacional de Microbio...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,[{'tag': None}],{'created_at': 'Fri Jan 31 22:19:44 +0000 2020...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fri Jan 31 23:58:59 +0000 2020,1223395282538369024,1223395282538369024,"RT @JaneLytv: 43. Zero Hedge, a pro-Trump webs...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,[{'tag': None}],{'created_at': 'Fri Jan 31 22:19:00 +0000 2020...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(df)

3700

In [7]:
print(len(df))
print(df.info())
df.describe()

3700
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700 entries, 0 to 3699
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 3700 non-null   object 
 1   id                         3700 non-null   int64  
 2   id_str                     3700 non-null   int64  
 3   text                       3700 non-null   object 
 4   source                     3700 non-null   object 
 5   truncated                  3700 non-null   bool   
 6   in_reply_to_status_id      168 non-null    float64
 7   in_reply_to_status_id_str  168 non-null    float64
 8   in_reply_to_user_id        179 non-null    float64
 9   in_reply_to_user_id_str    179 non-null    float64
 10  in_reply_to_screen_name    175 non-null    object 
 11  user                       3700 non-null   object 
 12  geo                        2 non-null      object 
 13  coordinates                2 non-null      

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,contributors,quote_count,reply_count,retweet_count,favorite_count,quoted_status_id,quoted_status_id_str
count,3.700000e+03,3.700000e+03,1.680000e+02,1.680000e+02,1.790000e+02,1.790000e+02,0.0,3700.000000,3700.000000,3700.000000,3700.000000,2.910000e+02,2.910000e+02
mean,1.223395e+18,1.223395e+18,1.223288e+18,1.223288e+18,3.499310e+17,3.499310e+17,NaN,0.077568,0.144324,0.737297,1.838649,1.215766e+18,1.215766e+18
std,2.501278e+11,2.501278e+11,2.966750e+14,2.966750e+14,4.920169e+17,4.920169e+17,NaN,1.236414,1.770484,12.989657,33.609435,6.267048e+16,6.267048e+16
min,1.223394e+18,1.223394e+18,1.220591e+18,1.220591e+18,1.917731e+06,1.917731e+06,NaN,0.000000,0.000000,0.000000,0.000000,6.098440e+17,6.098440e+17
25%,1.223395e+18,1.223395e+18,1.223315e+18,1.223315e+18,1.184868e+08,1.184868e+08,NaN,0.000000,0.000000,0.000000,0.000000,1.222953e+18,1.222953e+18
50%,1.223395e+18,1.223395e+18,1.223388e+18,1.223388e+18,1.385192e+09,1.385192e+09,NaN,0.000000,0.000000,0.000000,0.000000,1.223248e+18,1.223248e+18
75%,1.223395e+18,1.223395e+18,1.223393e+18,1.223393e+18,8.716089e+17,8.716089e+17,NaN,0.000000,0.000000,0.000000,0.000000,1.223360e+18,1.223360e+18
max,1.223395e+18,1.223395e+18,1.223395e+18,1.223395e+18,1.223048e+18,1.223048e+18,NaN,41.000000,82.000000,654.000000,1716.000000,1.223394e+18,1.223394e+18


# Exploratory Data Analysis

In [9]:
# Drop geo and controbuter data
df.drop(['geo', 'coordinates', 'place', 'contributors'], axis=1, inplace=True)